<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/mm-imdb(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %cd /content/Dataset(s)/mm-imdb/fused

In [1]:
!gdown https://drive.google.com/uc?id=1dxd2pySfCIDJYG7qMtuJre8ph068xc1X

Downloading...
From: https://drive.google.com/uc?id=1dxd2pySfCIDJYG7qMtuJre8ph068xc1X
To: /content/test.zip
100% 2.12G/2.12G [00:29<00:00, 73.0MB/s]


In [2]:
!gdown https://drive.google.com/uc?id=1sYR9EgHkM0oiGRQVlFQCyHO8kMRJ4ibQ

Downloading...
From: https://drive.google.com/uc?id=1sYR9EgHkM0oiGRQVlFQCyHO8kMRJ4ibQ
To: /content/test_label.txt
100% 777k/777k [00:00<00:00, 74.7MB/s]


In [3]:
!gdown https://drive.google.com/uc?id=1X4cmMYRjxXFomCJ1adMhPNMYtd4WeDHP

Downloading...
From: https://drive.google.com/uc?id=1X4cmMYRjxXFomCJ1adMhPNMYtd4WeDHP
To: /content/train_label.txt
100% 1.55M/1.55M [00:00<00:00, 188MB/s]


In [4]:
!gdown https://drive.google.com/uc?id=1yAeNEPXD3LNxmtSnC09jEg0vEDc1qiaq

Downloading...
From: https://drive.google.com/uc?id=1yAeNEPXD3LNxmtSnC09jEg0vEDc1qiaq
To: /content/train.zip
100% 4.21G/4.21G [00:35<00:00, 119MB/s]


In [ ]:
!unzip test.zip

In [ ]:
!unzip train.zip

# New Section

In [ ]:
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_file, root_dir, transform=None):
        self.data_file = data_file
        self.root_dir = root_dir
        self.transform = transform

        with open(data_file, 'r') as f:
            self.data = f.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        line = self.data[idx].strip().split('|')
        image_path = line[0].strip()
        image = Image.open(os.path.join(self.root_dir, image_path))

        labels = line[1].strip().split(',')
        labels = [label.strip() for label in labels]

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = CustomDataset(data_file='/content/test_label.txt', root_dir='/content/test', transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
with open('/content/test_label.txt', 'r') as f:
    lines = f.readlines()

unique_labels = set()
for line in lines:
    labels = line.strip().split('|')[1].strip().split(',')
    unique_labels.update(labels)

num_classes = len(unique_labels)
print(f"Number of classes: {num_classes}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define your ResNet-101 model
model = models.resnet101(pretrained=True)  # You can use a pre-trained model

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes),
    nn.Sigmoid()
)

In [ ]:
criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {epoch_loss:.4f}')

print('Training finished.')

# Now, you can use the trained model for evaluation and prediction
# Don't forget to create a DataLoader for your test/validation dataset and evaluate the model's performance.


# Training Dataset Preparation

In [7]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = "/content/train"
labels_file = '/content/train_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [2]:
len(image_file_paths)

46656

In [ ]:
# genre_labels

In [ ]:
# unique_labels_set = set()

# for labels in genre_labels:
#     unique_labels_set.update(labels)

# unique_labels = sorted(list(unique_labels_set))

In [ ]:
# unique_labels

In [ ]:
# len(unique_labels)

In [8]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [9]:
# Convert the defaultdict into a list of tuples
label_count_list = [(label, count) for label, count in label_counts.items()]

# Sort the list of tuples based on counts in descending order
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)

# Display the sorted list
for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

Drama: 25272
Comedy: 15324
Romance: 9678
Thriller: 9339
Crime: 6879
Action: 6465
Adventure: 4833
Horror: 4809
Documentary: 3702
Mystery: 3693
Sci-Fi: 3636
Fantasy: 3486
Family: 2934
War: 2418
Biography: 2364
History: 2040
Music: 1902
Animation: 1758
Musical: 1509
Western: 1269
Sport: 1137
Short: 843
Film-Noir: 606
News: 117
Talk-Show: 6
Reality-TV: 3
Total Labels:  26


In [10]:
min_label_count = 500
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [11]:
valid_labels, len(valid_labels)

(['Action',
  'Adventure',
  'Animation',
  'Biography',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Family',
  'Fantasy',
  'Film-Noir',
  'History',
  'Horror',
  'Music',
  'Musical',
  'Mystery',
  'Romance',
  'Sci-Fi',
  'Short',
  'Sport',
  'Thriller',
  'War',
  'Western'],
 23)

In [7]:
# filtered_image_paths = []
# filtered_genre_labels = []

# for image_path, labels in zip(image_file_paths, genre_labels):
#     valid_labels_for_sample = [label for label in labels if label in valid_labels]

#     if valid_labels_for_sample:
#         filtered_image_paths.append(image_path)
#         filtered_genre_labels.append(valid_labels_for_sample)

In [ ]:
# len(filtered_image_paths)

In [ ]:
# len(filtered_genre_labels)

In [12]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    multi_hot_labels.append(multi_hot)

In [13]:
multi_hot_labels[1]

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [14]:
genre_labels[1]

['Crime', 'Drama', 'Thriller']

In [15]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [16]:
# Create a custom dataset
train_dataset = CustomMultiLabelDataset(image_file_paths, multi_hot_labels, transform=transform)

In [17]:
len(train_dataset)

46656

In [18]:
train_dataset[10]

(tensor([[[-0.4739, -1.2103, -1.2617,  ..., -1.2617, -1.2103, -0.4568],
          [-0.7137, -2.0494, -2.0837,  ..., -2.0665, -2.0323, -0.7137],
          [-0.6281, -2.1179, -2.1179,  ..., -2.1179, -2.1008, -0.6281],
          ...,
          [ 0.5193,  1.7865,  1.7352,  ...,  1.9920,  2.2489,  0.6221],
          [ 0.5022,  1.7865,  1.7009,  ...,  2.0263,  2.2318,  0.6563],
          [ 0.2624,  1.0159,  1.0159,  ...,  1.1015,  1.2899,  0.3138]],
 
         [[-0.3550, -1.1078, -1.1604,  ..., -1.1604, -1.1078, -0.3375],
          [-0.6001, -1.9657, -2.0007,  ..., -1.9832, -1.9482, -0.6001],
          [-0.5126, -2.0357, -2.0357,  ..., -2.0357, -2.0182, -0.5126],
          ...,
          [ 0.6954,  2.0434,  2.0609,  ...,  2.1134,  2.3936,  0.6954],
          [ 0.8004,  2.1660,  2.0959,  ...,  2.2010,  2.4111,  0.7654],
          [ 0.4503,  1.2556,  1.3081,  ...,  1.3256,  1.4657,  0.4678]],
 
         [[-0.1312, -0.8807, -0.9330,  ..., -0.9330, -0.8807, -0.1138],
          [-0.3753, -1.7347,

# Test Dataset Preparation

In [19]:
import os
test_image_file_paths = []
test_genre_labels = []

image_folder_add = "/content/test"
labels_file = '/content/test_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        test_image_file_paths.append(image_path)
        test_genre_labels.append(labels)

In [ ]:
# unique_labels_set = set()

# for labels in test_genre_labels:
#     unique_labels_set.update(labels)

# unique_labels = sorted(list(unique_labels_set))

In [ ]:
# unique_labels

In [20]:
test_multi_hot_labels = []

for labels in test_genre_labels:
    test_multi_hot = [1 if label in labels else 0 for label in valid_labels]
    test_multi_hot_labels.append(test_multi_hot)

In [21]:
test_dataset = CustomMultiLabelDataset(test_image_file_paths, test_multi_hot_labels, transform=transform)

In [22]:
len(test_dataset)

23397

# Training

In [23]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = models.resnet101(pretrained=True)  # You can use a pre-trained model

# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 512),
#     nn.ReLU(),
#     nn.Linear(512, len(unique_labels)),
#     nn.Sigmoid()
# )

model = models.resnet101(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(valid_labels))
model.to(device)

In [26]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    if epoch > 15:
      save_dir = "/content/Model/Models-Train-21/"
      os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
      model_name = str(epoch) + "_model.pth"
      save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
      torch.save(model.state_dict(), save_path)

    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            # loss = criterion(outputs, labels.float())

            # Apply a threshold (e.g., 0.5) to convert logits to binary predictions
            predictions.extend((outputs > 0.5).int().cpu().numpy())
            true_labels.extend(labels.int().cpu().numpy())

    # Calculate evaluation metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(classification_report(true_labels, predictions))

Epoch 1/30, Loss: 0.2787469495675528


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.06282856776509808
Precision: 0.12958421262729813
Recall: 0.02504954076161011
F1-Score: 0.029436553952304944
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3132
           1       0.00      0.00      0.00      2463
           2       0.00      0.00      0.00       918
           3       0.00      0.00      0.00      1233
           4       0.66      0.03      0.05      7833
           5       0.00      0.00      0.00      3489
           6       0.00      0.00      0.00      1887
           7       0.64      0.52      0.57     12426
           8       0.00      0.00      0.00      1554
           9       0.00      0.00      0.00      1755
          10       0.00      0.00      0.00       306
          11       0.00      0.00      0.00      1035
          12       0.50      0.00      0.00      2475
          13       0.00      0.00      0.00       933
          14       0.00      0.00      0.00       759
          15   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/30, Loss: 0.2540216508879747


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 10/30, Loss: 0.158401901145122
Accuracy: 0.16249946574347138
Precision: 0.6594604718642151
Recall: 0.2935672087311426
F1-Score: 0.37855491812868813
              precision    recall  f1-score   support

           0       0.75      0.23      0.35      3132
           1       0.66      0.27      0.39      2463
           2       0.71      0.27      0.40       918
           3       0.54      0.09      0.15      1233
           4       0.66      0.61      0.63      7833
           5       0.77      0.24      0.36      3489
           6       0.80      0.58      0.67      1887
           7       0.77      0.66      0.71     12426
           8       0.68      0.34      0.45      1554
           9       0.63      0.21      0.32      1755
          10       0.36      0.04      0.07       306
          11       0.55      0.10      0.17      1035
          12       0.83      0.40      0.54      2475
          13       0.68      0.28      0.40       933
          14       0.42      0.09  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 11/30, Loss: 0.14505958274538297
Accuracy: 0.15997777492840962
Precision: 0.6434594222007504
Recall: 0.3029385588413977
F1-Score: 0.3752207918116456
              precision    recall  f1-score   support

           0       0.71      0.26      0.38      3132
           1       0.72      0.16      0.26      2463
           2       0.82      0.06      0.10       918
           3       0.35      0.20      0.25      1233
           4       0.73      0.42      0.54      7833
           5       0.67      0.45      0.54      3489
           6       0.72      0.66      0.69      1887
           7       0.68      0.82      0.74     12426
           8       0.78      0.22      0.34      1554
           9       0.74      0.06      0.11      1755
          10       0.26      0.13      0.17       306
          11       0.35      0.25      0.29      1035
          12       0.86      0.30      0.45      2475
          13       0.62      0.37      0.46       933
          14       0.43      0.10 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# New Section

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

save_dir = "/content/Model/Models-Train-1"
load_path = os.path.join(save_dir, '8_model.pth')


model = models.resnet101(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, len(unique_labels)),
    nn.Sigmoid()
)
model.load_state_dict(torch.load(load_path))
model.eval()
model.to(device)

In [ ]:
# Initialize lists to store predicted and ground truth labels
predicted_labels = []
true_labels = []

# Iterate through the test set and make predictions
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted_labels.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
predicted_labels[0]

In [ ]:
threshold = 0.5

binary_predictions = []

for sample_logits in predicted_labels:
    binary_sample_predictions = [1 if value >= threshold else 0 for value in sample_logits]
    binary_predictions.append(binary_sample_predictions)

In [ ]:
binary_predictions[1000]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(true_labels, binary_predictions)
precision = precision_score(true_labels, binary_predictions, average='macro')
recall = recall_score(true_labels, binary_predictions, average='macro')
f1 = f1_score(true_labels, binary_predictions, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(classification_report(true_labels, binary_predictions))